In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

load data

In [620]:
merged_df = pd.read_csv('data/main/data_set.csv')

rename for ols formular

In [621]:
merged_df = merged_df.rename(columns={'debt/equity_ratio_2019' : 'debtxtoxequity_ratio_2019', 
                                        'debt/asset_ratio_2019' : 'debtxtoxassest_ratio_2019',
                                        'debt/equity_ratio_2020' : 'debtxtoxequity_ratio_2020', 
                                        'debt/asset_ratio_2020' : 'debtxtoxassest_ratio_2020',
                                        'debt/equity_ratio_2021' : 'debtxtoxequity_ratio_2021', 
                                        'debt/asset_ratio_2021' : 'debtxtoxassest_ratio_2021',
                                        })

In [622]:
ratios = ['cash_ratio','quick_ratio','current_ratio','debtxtoxequity_ratio','equity_ratio','debtxtoxassest_ratio']
types = ['aid_grant','aid_loan']

In [616]:
results_dict = {}
#loop over year
for year in [2020,2021]:

    for id, type in enumerate(types, start=1):
        type_inverse_year = types[id-2] + '_' + str(year)
        type_year = type + '_' + str(year)
        data_unconfounded = merged_df[merged_df[type_inverse_year].isnull()]

        #loop over ratio
        if year == 2021: # Remove companies that received other aid due to possible confounding
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_grant'+'_'+ str(year-1))].isnull()] ###
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_loan'+'_'+ str(year-1))].isnull()] ###
        
        results = {}
        

        for ratio in ratios:
            ratio_year_prior = ratio + '_' + str(year-1)
            ratio_year = ratio + '_' + str(year)

            data_dd = data_unconfounded[[ratio_year_prior , ratio_year, type_year]]
            data_dd['treatment_group'] = np.where(data_dd[type_year].isna() , 0, 1)
            data_dd = data_dd.drop(type_year, axis=1)
            data_dd.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_dd = data_dd.dropna()

            #remove outliers
            data_dd = data_dd[data_dd[ratio_year_prior].between(*data_dd[ratio_year_prior].quantile([0.05, 0.95]).tolist())]
            data_dd = data_dd[data_dd[ratio_year].between(*data_dd[ratio_year].quantile([0.05, 0.95]).tolist())]

            # data before the treatment
            df_before = data_dd[[ratio_year_prior, 'treatment_group']]
            df_before['t'] = 0
            df_before.columns = [ratio, 'treatment_group', 'time']

            # data after the treatment
            df_after = data_dd[[ratio_year, 'treatment_group']]
            df_after['t'] = 1
            df_after.columns = [ratio, 'treatment_group', 'time']

            # data for regression
            df_reg = pd.concat([df_before, df_after])

            # create the interaction 
            df_reg['interaction'] = df_reg.treatment_group * df_reg.time

            # drop extrem outliers
            df_reg = df_reg[df_reg[ratio] > 0 ]
            df_reg = df_reg.drop(df_reg[df_reg[ratio] > df_reg[ratio].quantile(.95)].index)
            

            from statsmodels.formula.api import ols

            formular = ratio + ' ~ treatment_group + time + interaction' 
            ols = ols(formular, data=df_reg).fit()
            cov = ols.params['interaction'].round(4)
            pv =  ols.pvalues['interaction'].round(3)
            
            
            if pv >= 0.1:
                pv = ("%.3f" % pv)           
                pv = "\space\space\space\space("+str(pv)+")"
            elif (pv < 0.1) and ((pv > 0.05)):
                pv = ("%.3f" % pv)           
                pv = "*\space\space\space("+str(pv)+")" 
            elif (pv < 0.05) and ((pv > 0.01)):
                pv = ("%.3f" % pv)           
                pv = "**\space\space("+str(pv)+")"
            elif pv < 0.01:
                pv = ("%.3f" % pv)           
                pv = "***\space("+str(pv)+")"

            ratio = ratio.replace("_", " ").replace("x", " ")
            results[ratio] = ''.join((str(cov),pv))

        results_dict[type_year] = results

format output

In [617]:
df_results = pd.DataFrame.from_dict(results_dict, orient='index')
df_results.index.name='aid_instrument_year'
df_results.reset_index(inplace=True)
new_col = df_results['aid_instrument_year'].str.split('_',expand=True)
df_results['instrument'] = new_col[1]
df_results['year'] = new_col[2]
df_results.drop(columns =["aid_instrument_year"], inplace = True)
df_results.set_index(['year', 'instrument'],inplace=True)
df_results

cash ratio  \
year instrument                                           
2020 grant       -0.0089\space\space\space\space(0.270)   
     loan                        0.0527***\space(0.000)   
2021 grant                       0.0768***\space(0.000)   
     loan                        0.0351***\space(0.000)   

                                            quick ratio  \
year instrument                                           
2020 grant       -0.0937\space\space\space\space(0.221)   
     loan                        0.1085***\space(0.000)   
2021 grant        0.0913\space\space\space\space(0.478)   
     loan         0.0929\space\space\space\space(0.252)   

                                          current ratio  \
year instrument                                           
2020 grant       -0.0825\space\space\space\space(0.330)   
     loan                        0.1271***\space(0.000)   
2021 grant        0.0476\space\space\space\space(0.736)   
     loan              0.1828*\space\space\space(0.064)   

                                   debt to equity ratio  \
year instrument                                           
2020 grant             0.2478*\space\space\space(0.089)   
     loan                        0.7306***\space(0.000)   
2021 grant       -0.1592\space\space\space\space(0.185)   
     loan                   0.5043**\space\space(0.016)   

                                           equity ratio  \
year instrument                                           
2020 grant       -0.0112\space\space\space\space(0.371)   
     loan                       -0.0486***\space(0.000)   
2021 grant        0.0108\space\space\space\space(0.465)   
     loan                        -0.037***\space(0.004)   

                              debt to assest ratio  
year instrument                                     
2020 grant                  0.0412***\space(0.006)  
     loan                   0.0715***\space(0.000)  
2021 grant       -0.0302*\space\space\space(0.079)  
     loan                   0.0526***\space(0.002)

save data frame to latex

In [618]:
df_results.transpose().stack()

year                                                               2020  \
                     instrument                                           
cash ratio           grant       -0.0089\space\space\space\space(0.270)   
                     loan                        0.0527***\space(0.000)   
quick ratio          grant       -0.0937\space\space\space\space(0.221)   
                     loan                        0.1085***\space(0.000)   
current ratio        grant       -0.0825\space\space\space\space(0.330)   
                     loan                        0.1271***\space(0.000)   
debt to equity ratio grant             0.2478*\space\space\space(0.089)   
                     loan                        0.7306***\space(0.000)   
equity ratio         grant       -0.0112\space\space\space\space(0.371)   
                     loan                       -0.0486***\space(0.000)   
debt to assest ratio grant                       0.0412***\space(0.006)   
                     loan                        0.0715***\space(0.000)   

year                                                               2021  
                     instrument                                          
cash ratio           grant                       0.0768***\space(0.000)  
                     loan                        0.0351***\space(0.000)  
quick ratio          grant        0.0913\space\space\space\space(0.478)  
                     loan         0.0929\space\space\space\space(0.252)  
current ratio        grant        0.0476\space\space\space\space(0.736)  
                     loan              0.1828*\space\space\space(0.064)  
debt to equity ratio grant       -0.1592\space\space\space\space(0.185)  
                     loan                   0.5043**\space\space(0.016)  
equity ratio         grant        0.0108\space\space\space\space(0.465)  
                     loan                        -0.037***\space(0.004)  
debt to assest ratio grant            -0.0302*\space\space\space(0.079)  
                     loan                        0.0526***\space(0.002)

In [619]:
with open('/Users/marco/Library/CloudStorage/OneDrive-HertieSchool/Thesis/GitHub/thesis/paper/Tables/table_did.tex', 'w') as f:
     f.write(df_results.transpose().stack().to_latex(index=True, bold_rows=True, column_format='llrr',multirow=True ,formatters={'white-space': 'pre'}, escape=False))

/var/folders/nr/94zdnpbn2xj3pwc_d9lfk0cw0000gn/T/ipykernel_48019/2348292745.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df_results.transpose().stack().to_latex(index=True, bold_rows=True, column_format='llrr',multirow=True ,formatters={'white-space': 'pre'}, escape=False))


#### Report for all covariates

In [641]:
results_dict= {}

for year in [2020,2021]:

    for id, type in enumerate(types, start=1):
        type_inverse_year = types[id-2] + '_' + str(year)
        type_year = type + '_' + str(year)
        data_unconfounded = merged_df[merged_df[type_inverse_year].isnull()]

        #loop over ratio
        if year == 2021: # Remove companies that received other aid due to possible confounding
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_grant'+'_'+ str(year-1))].isnull()] ###
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_loan'+'_'+ str(year-1))].isnull()] ###
        
        results = {}
        

        for ratio in ratios:
            results_sub = {}
            ratio_year_prior = ratio + '_' + str(year-1)
            ratio_year = ratio + '_' + str(year)

            data_dd = data_unconfounded[[ratio_year_prior , ratio_year, type_year]]
            data_dd['treatment_group'] = np.where(data_dd[type_year].isna() , 0, 1)
            data_dd = data_dd.drop(type_year, axis=1)
            data_dd.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_dd = data_dd.dropna()

            #remove outliers
            data_dd = data_dd[data_dd[ratio_year_prior].between(*data_dd[ratio_year_prior].quantile([0.05, 0.95]).tolist())]
            data_dd = data_dd[data_dd[ratio_year].between(*data_dd[ratio_year].quantile([0.05, 0.95]).tolist())]

            # data before the treatment
            df_before = data_dd[[ratio_year_prior, 'treatment_group']]
            df_before['t'] = 0
            df_before.columns = [ratio, 'treatment_group', 'time']

            # data after the treatment
            df_after = data_dd[[ratio_year, 'treatment_group']]
            df_after['t'] = 1
            df_after.columns = [ratio, 'treatment_group', 'time']

            # data for regression
            df_reg = pd.concat([df_before, df_after])

            # create the interaction 
            df_reg['interaction'] = df_reg.treatment_group * df_reg.time

            # drop extrem outliers
            df_reg = df_reg[df_reg[ratio] > 0 ]
            df_reg = df_reg.drop(df_reg[df_reg[ratio] > df_reg[ratio].quantile(.95)].index)
            

            from statsmodels.formula.api import ols

            formular = ratio + ' ~ treatment_group + time + interaction' 
            ols = ols(formular, data=df_reg).fit()

            cov = ols.params['interaction'].round(4)
            pv =  ols.pvalues['interaction'].round(3)

            cov_time = ols.params['time'].round(4)
            pv_time =  ols.pvalues['time'].round(3)

            cov_treat = ols.params['treatment_group'].round(4)
            pv_treat =  ols.pvalues['treatment_group'].round(3)
            

            if pv >= 0.1:
                pv = ("%.3f" % pv)           
                pv = " ("+str(pv)+")"
            elif (pv < 0.1) and (pv > 0.05):
                pv = ("%.3f" % pv)           
                pv = "* ("+str(pv)+")"
            elif (pv < 0.05) and (pv > 0.01):
                pv = ("%.3f" % pv)           
                pv = "** ("+str(pv)+")"
            elif pv < 0.01:
                pv = ("%.3f" % pv)           
                pv = "*** ("+str(pv)+")"

            if pv_time >= 0.1:
                pv_time = ("%.3f" % pv_time)           
                pv_time = " ("+str(pv_time)+")"
            elif (pv_time < 0.1) and (pv_time > 0.05):
                pv_time = ("%.3f" % pv_time)           
                pv_time = "* ("+str(pv_time)+")"
            elif (pv_time < 0.05) and (pv_time > 0.01):
                pv_time = ("%.3f" % pv_time)           
                pv_time = "** ("+str(pv_time)+")"
            elif pv_time < 0.01:
                pv_time = ("%.3f" % pv_time)           
                pv_time = "*** ("+str(pv_time)+")"

            if pv_treat >= 0.1:
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = " ("+str(pv_treat)+")"
            elif (pv_treat < 0.1) and (pv_treat > 0.05):
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = "* ("+str(pv_treat)+")"
            elif (pv_treat < 0.05) and (pv_treat > 0.01):
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = "** ("+str(pv_treat)+")"
            elif pv_treat < 0.01:
                pv_treat = ("%.3f" % pv_treat)           
                pv_treat = "*** ("+str(pv_treat)+")"

            ratio = ratio.replace("_", " ").replace("x", " ").replace(" ratio", "")
            results_sub['aid'] = ''.join((str(cov_treat),pv_treat))
            results_sub['post'] = ''.join((str(cov_time),pv_time))
            results_sub['aid*post'] =''.join((str(cov),pv))
            results[ratio] = results_sub
    
        results_dict[type_year] = results

In [635]:
reformed_dict = {}
for outerKey, innerDict in results_dict.items():
    for innerKey, values in innerDict.items():
        reformed_dict[(outerKey,
                       innerKey)] = values
df_results_all = pd.DataFrame(reformed_dict)
df_results_all
  

aid_grant_2020                                        \
                       cash              quick            current   
aid       0.0283*** (0.000)  0.2112*** (0.000)     0.0655 (0.274)   
post        -0.0023 (0.322)   0.321*** (0.000)  0.3501*** (0.000)   
aid*post    -0.0089 (0.270)    -0.0937 (0.221)    -0.0825 (0.330)   

                                                                     \
              debt to equity             equity      debt to assest   
aid       -0.2703*** (0.009)     0.0095 (0.268)  -0.0347*** (0.001)   
post         -0.0676 (0.111)  0.0124*** (0.001)     -0.0047 (0.295)   
aid*post     0.2478* (0.089)    -0.0112 (0.371)   0.0412*** (0.006)   

               aid_loan_2020                                          \
                        cash               quick             current   
aid       -0.0406*** (0.000)  -0.2065*** (0.000)  -0.1688*** (0.000)   
post         -0.0001 (0.966)   0.2744*** (0.000)   0.3123*** (0.000)   
aid*post   0.0527*** (0.000)   0.1085*** (0.000)   0.1271*** (0.000)   

                             ...    aid_grant_2021                    \
             debt to equity  ...           current    debt to equity   
aid       0.3667*** (0.000)  ...  0.2055** (0.039)  -0.1569* (0.065)   
post         -0.055 (0.291)  ...    0.0768 (0.544)    0.0167 (0.877)   
aid*post  0.7306*** (0.000)  ...    0.0476 (0.736)   -0.1592 (0.185)   

                                                 aid_loan_2021  \
                  equity    debt to assest                cash   
aid       0.0123 (0.238)  -0.0231* (0.058)  -0.0319*** (0.000)   
post         0.0 (0.998)    0.0135 (0.385)   -0.0153** (0.027)   
aid*post  0.0108 (0.465)  -0.0302* (0.079)   0.0351*** (0.000)   

                                                                     \
                       quick             current     debt to equity   
aid       -0.2017*** (0.000)  -0.2285*** (0.001)  0.4226*** (0.005)   
post          0.0715 (0.269)      0.1031 (0.188)     0.0749 (0.656)   
aid*post      0.0929 (0.252)     0.1828* (0.064)   0.5043** (0.016)   

                                                
                     equity     debt to assest  
aid        -0.0159* (0.082)  0.0379*** (0.001)  
post        -0.0007 (0.947)     0.0165 (0.219)  
aid*post  -0.037*** (0.004)  0.0526*** (0.002)  

[3 rows x 24 columns]

In [636]:
df_results_all = pd.DataFrame.from_dict(reformed_dict, orient='index')

df_results_all.reset_index(inplace=True)
df_results_all = df_results_all.rename(columns={'level_0' : 'aid_type_year', 
                                        'level_1' : 'ratio'})

new_col = df_results_all['aid_type_year'].str.split('_',expand=True)
df_results_all['type'] = new_col[1]
df_results_all['year'] = new_col[2]
df_results_all.drop(columns =["aid_type_year"], inplace = True)
df_results_all.set_index(['type','year', 'ratio'],inplace=True)
df_results_all

aid               post  \
type  year ratio                                                   
grant 2020 cash             0.0283*** (0.000)    -0.0023 (0.322)   
           quick            0.2112*** (0.000)   0.321*** (0.000)   
           current             0.0655 (0.274)  0.3501*** (0.000)   
           debt to equity  -0.2703*** (0.009)    -0.0676 (0.111)   
           equity              0.0095 (0.268)  0.0124*** (0.001)   
           debt to assest  -0.0347*** (0.001)    -0.0047 (0.295)   
loan  2020 cash            -0.0406*** (0.000)    -0.0001 (0.966)   
           quick           -0.2065*** (0.000)  0.2744*** (0.000)   
           current         -0.1688*** (0.000)  0.3123*** (0.000)   
           debt to equity   0.3667*** (0.000)     -0.055 (0.291)   
           equity            -0.0058* (0.069)  0.0126*** (0.000)   
           debt to assest   0.0398*** (0.000)    -0.0013 (0.751)   
grant 2021 cash             -0.0159** (0.033)  -0.0199** (0.037)   
           quick            0.2879*** (0.002)     0.0147 (0.899)   
           current           0.2055** (0.039)     0.0768 (0.544)   
           debt to equity    -0.1569* (0.065)     0.0167 (0.877)   
           equity              0.0123 (0.238)        0.0 (0.998)   
           debt to assest    -0.0231* (0.058)     0.0135 (0.385)   
loan  2021 cash            -0.0319*** (0.000)  -0.0153** (0.027)   
           quick           -0.2017*** (0.000)     0.0715 (0.269)   
           current         -0.2285*** (0.001)     0.1031 (0.188)   
           debt to equity   0.4226*** (0.005)     0.0749 (0.656)   
           equity            -0.0159* (0.082)    -0.0007 (0.947)   
           debt to assest   0.0379*** (0.001)     0.0165 (0.219)   

                                     aid*post  
type  year ratio                               
grant 2020 cash               -0.0089 (0.270)  
           quick              -0.0937 (0.221)  
           current            -0.0825 (0.330)  
           debt to equity     0.2478* (0.089)  
           equity             -0.0112 (0.371)  
           debt to assest   0.0412*** (0.006)  
loan  2020 cash             0.0527*** (0.000)  
           quick            0.1085*** (0.000)  
           current          0.1271*** (0.000)  
           debt to equity   0.7306*** (0.000)  
           equity          -0.0486*** (0.000)  
           debt to assest   0.0715*** (0.000)  
grant 2021 cash             0.0768*** (0.000)  
           quick               0.0913 (0.478)  
           current             0.0476 (0.736)  
           debt to equity     -0.1592 (0.185)  
           equity              0.0108 (0.465)  
           debt to assest    -0.0302* (0.079)  
loan  2021 cash             0.0351*** (0.000)  
           quick               0.0929 (0.252)  
           current            0.1828* (0.064)  
           debt to equity    0.5043** (0.016)  
           equity           -0.037*** (0.004)  
           debt to assest   0.0526*** (0.002)

In [637]:
with open('/Users/marco/Library/CloudStorage/OneDrive-HertieSchool/Thesis/GitHub/thesis/paper/Tables/table_did_all.tex', 'w') as f:
     f.write(df_results_all.to_latex())

/var/folders/nr/94zdnpbn2xj3pwc_d9lfk0cw0000gn/T/ipykernel_48019/3209959191.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df_results_all.to_latex())
